# AFL Predictions
The file we will run to get each rounds predictions!

In [1]:
#packages
import polars as pl
import pickle
import numpy as np
import pandas as pd
#Get Update function
from ipynb.fs.defs.AFLupdate import updateData, getCurrentRoundNumbers
#Get upcoming fixture
from ipynb.fs.defs.AFLfixture import createFixture

pl.Config.set_fmt_str_lengths(150);
pl.Config.set_tbl_cols(40);
pl.Config.set_tbl_rows(50);


/home/erin/Documents/AFL tipping/afl-env/lib/python3.12/site-packages/rpy2/robjects/lib/dplyr.py:27: UserWarning: This was designed against dplyr versions starting with 1.0 but you have 1.1.4
  warnings.warn(


In [2]:
#Run updates
updateData()

ℹ Returning data for "All Rounds, 2025"
✔ Returning data for "All Rounds, 2025" [1.1s]

ℹ Returning data for "All Rounds, 2025"
✔ Returning data for "All Rounds, 2025" s]

13


/home/erin/Documents/AFL tipping/AFLmodel/AFLupdate.ipynb:357: MapWithoutReturnDtypeWarning: Calling `map_elements` without specifying `return_dtype` can lead to unpredictable results. Specify `return_dtype` to silence this warning.
  {


Data Updated for New Round


In [3]:
#get the fixture 
fixture = createFixture()

ℹ Returning data for "All Rounds, 2025"
✔ Returning data for "All Rounds, 2025" s]

ℹ Returning data for "Round 14, 2025"
✔ Returning data for "Round 14, 2025" s]

ℹ Returning data for "All Rounds, 2025"
✔ Returning data for "All Rounds, 2025" s]

ℹ Returning data for "Round 13, 2025"
✔ Returning data for "Round 13, 2025" s]

ℹ Returning data for "All Rounds, 2025"
✔ Returning data for "All Rounds, 2025" s]

ℹ Returning data for "All Rounds, 2025"
✔ Returning data for "All Rounds, 2025" s]

ℹ Returning data for "All Rounds, 2025"
✔ Returning data for "All Rounds, 2025" s]

ℹ Fetching match ids
✔ Fetching match ids s]

ℹ Finding player stats for 216 matches.
✔ Finding player stats for 216 matches. [46.1s]

ℹ Fetching match ids
✔ Fetching match ids s]

ℹ Finding player stats for 207 matches.
✔ Finding player stats for 207 matches. [29.7s]



In [4]:
#put everything in order
fixtureReady = (fixture
    .select('home.bye.flag', 'home.ladder.position', 'home.elo.score', 'home.venue.portion', 'home.day.last.match', 'home.attack.t-1', 'home.attack.t-2',
                   'home.attack.t-3', 'home.attack.t-4', 'home.attack.t-5', 'home.defense.t-1', 'home.defense.t-2', 'home.defense.t-3', 'home.defense.t-4', 'home.defense.t-5', 'home.opposition.strength',
                  'home.team.playerPoints', 'away.bye.flag', 'away.ladder.position', 'away.elo.score', 'away.venue.portion', 'away.day.last.match', 'away.attack.t-1', 'away.attack.t-2',
                   'away.attack.t-3', 'away.attack.t-4', 'away.attack.t-5', 'away.defense.t-1', 'away.defense.t-2', 'away.defense.t-3', 'away.defense.t-4', 'away.defense.t-5', 'away.opposition.strength',
                   'away.team.playerPoints')   
    .unique(maintain_order=True)         
    )
#get list of games for presentation of fixture predictions
predictionReady = (fixture
                    .select('home.team.name', 'away.team.name'))

In [5]:
#if fixture exists i.e no issues have been introduced, go ahead and use our models to get our predictions
modelWin = pickle.load(open('Data/winner.sav', 'rb'))

predictWinner = modelWin.predict(fixtureReady).tolist()
probsWinner = modelWin.predict_proba(fixtureReady).tolist()

#create the winner portion of the prediction df
predictionDF = (predictionReady
                 #place in results
                 .with_columns(pl.Series(predictWinner).alias('Predictions'),
                               pl.Series(probsWinner).alias('Probs'))
                 #Winner probs
                 .with_columns(((pl.col('Probs').list.get(pl.col('Predictions')))*100).round(2).alias('Win Prob'))
                 .with_columns(pl.when(pl.col('Predictions') == 1).then(pl.col('home.team.name')).otherwise(pl.col('away.team.name')).alias('Predicted Winner'))
                 #Add current round
                 .with_columns(pl.lit(getCurrentRoundNumbers()).alias('Round Number'))
                 #rename
                 .rename({'home.team.name' : 'Home Team',
                          'away.team.name' : 'Away Team'})
                 #keep only wants required
                 .select('Round Number', 'Home Team', 'Away Team', 'Predicted Winner', 'Win Prob')
                )


ℹ Returning data for "All Rounds, 2025"
✔ Returning data for "All Rounds, 2025" s]



In [6]:
#Get the margin
#load the model in
model = pickle.load(open('Data/margin.sav', 'rb'))

#And the win probablities to fixture
fixtureReady[['homeProb', 'awayProb']] = probsWinner
#predict the fixture
predictMargin = model.predict(fixtureReady).tolist()

#Add to prediction df
predictionDF = (predictionDF.with_columns(pl.Series(predictMargin).round(0).cast(pl.Int32).alias('Margin')))


In [7]:
predictionDF

Round Number,Home Team,Away Team,Predicted Winner,Win Prob,Margin
i32,str,str,str,f64,i32
14,"""St Kilda""","""Western Bulldogs""","""St Kilda""",60.2,21
14,"""Hawthorn""","""Adelaide Crows""","""Hawthorn""",50.98,25
14,"""Brisbane Lions""","""GWS Giants""","""Brisbane Lions""",72.88,25
14,"""Essendon""","""Geelong Cats""","""Essendon""",51.5,21
14,"""North Melbourne""","""Fremantle""","""Fremantle""",61.1,21
14,"""Port Adelaide""","""Melbourne""","""Port Adelaide""",55.4,26
14,"""West Coast Eagles""","""Carlton""","""West Coast Eagles""",52.0,21


In [8]:
#Write to folder
#Individual round tips
predictionDF.write_csv(fr'Tips/2025/Round_{getCurrentRoundNumbers()}.csv')

#Append to all_tips
with open("Tips/2025/All_Tips.csv", mode="a") as f:
   predictionDF.write_csv(f, include_header=False)

ℹ Returning data for "All Rounds, 2025"
✔ Returning data for "All Rounds, 2025" s]

